In [1]:
#install.packages("lubridate")
#install.packages("irr")
#install.packages("readr")
library ('irr')
library("readr")
library("tidyr")
library ('tidyverse')
library ("icr")
library('caret')
library('plyr') #for mapping fct

Loading required package: lpSolve

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.2     v dplyr   1.0.1
v tibble  3.0.3     v stringr 1.4.0
v purrr   0.3.4     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: lattice

Registered S3 method overwritten by 'caret':
  method    from
  print.icr icr 


Attaching package: 'caret'


The following object is masked from 'package:purrr':

    lift


------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: 'plyr'


The following objects are masked

In [2]:
#TODO ADD DETAILS
#Kappa it is unweighed because it is not worse having a mismatch between e.g. location 1 and 2 in comparison between other locations
#level of agreement is percentage of same data entry; kappa considers the fact that some might be the same due to chance    
#agree with tolerance 0: o not accept deviation from 0, there must be either perfect match or else it is not matching at all, we do not consider a difference of 1 zone level less mismatching than a difference of 2 or 3 zone levels 

In [3]:
path_records = "R:\\VPHI\\Welfare\\2- Research Projects\\OFHE2.OriginsE2\\Ganter system validation\\RECORDS"
path_analysis = "R:\\VPHI\\Welfare\\2- Research Projects\\OFHE2.OriginsE2\\Ganter system validation\\Analysis\\Results"
sink(file.path(path_analysis,'GantnerAnalysisSummary.txt'))

# Video & Focals & Scan observations systems statistics

In [4]:
c_uniform_naming_from = c('Tier 1', 'Tier 3 Rampe + Nestbox', 'Tier 2 (mini 12)','Tier 2 (mini 11)', 'Wintergarten', 
                          'Tier 2 (mini 10)','Tier 2 + untere Stange', 'Tier 4 + obere Stange','Tier 2 (mini 4)', 
                          'Tier 2 (mini 5)', 'Tier 2 (mini 3)','Tier 2 (mini 8)','Tier 2 (mini 9)','Tier 2',
                          'Tier 1','Tier 3', 'Tier 4','1','2','3','4','Wg','N/A', NA)
c_uniform_naming_to = c('zone_1', 'zone_3','zone_2','zone_2','zone_wg', 'zone_2','zone_2', 'zone_4','zone_2', 'zone_2', 
                        'zone_2','zone_2', 'zone_2','zone_2','zone_1','zone_3','zone_4','zone_1','zone_2','zone_3','zone_4',
                        'zone_wg',NA,NA)

In [5]:
c_csv = c('scan_automatic_matching.csv','ValidationAllFocals.csv','VideoObservation\\VideoObservationAllStudents.csv')
for (x in c_csv){
        
    #open file
    df = read_delim(file.path(path_records,x),';',trim_ws = TRUE)
    
    #map values to uniform way of naming
    df$obsZone = mapvalues(df$obsZone, from=c_uniform_naming_from, to=c_uniform_naming_to)
    df$trackZone = mapvalues(df$trackZone, from=c_uniform_naming_from, to=c_uniform_naming_to)

    #remove nan. focals/video/scans: missed trackzone has the name of the zone where the animal is in. If we dont have 
    #(and have NA) it might be due to something else
    print(dim(df))
    df = df[complete.cases(df[ , c('obsZone','trackZone')]),]
    print(dim(df))
    
    #into factor for confusion matrix and test
    df[c('obsZone','trackZone')] = lapply(df[c('obsZone','trackZone')], factor)
    
    #for video observation remove duplicates rows that have same outcome for same observations but different workers
    #keep only the rows that are true are false: 0,1
    #remove nabila
    if (x=='VideoObservation\\VideoObservationAllStudents.csv'){
        print(dim(df))
        df = subset(df, (df$VideoAnalyse == 0 | df$VideoAnalyse == 1) & (df$worker!='nabila')) 
        print(dim(df))
        #df$Heure = as.POSIXct(df$Heure,format="%H:%M:%S")
        #df[c('BirdID','VideoAnalyse')] = lapply(df[c('BirdID','VideoAnalyse')], factor)
        df = distinct(df, Date, Heure, trackZone, BirdID, VideoAnalyse, .keep_all = TRUE)
        #distinct(df, Date, .keep_all = TRUE)
        print(dim(df))
    }

    ################ reliability ################
    cat('------------------------------------------------------------------\n',
        file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat(paste0('-----------------',x,'reliability -------------------\n'),
        file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat('------------------------------------------------------------------\n',
        file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)

    #general reliability
    cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat('-------all pen --------\n', file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)    #reliability all systems together
    capture.output(kappa2(df[,c('obsZone','trackZone')], "unweighted"),
                   file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    capture.output(agree(df[,c('obsZone','trackZone')], tolerance=0),
                   file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)

    #reliability for each systems separately
    c_pens = list(c(3,4,5),c(8,9),c(10,11,12))
    for (c_pen in c_pens){
        df_ = subset(df, df$Pen %in% c_pen)
        cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        cat(paste0('pen: \n ',c_pen,'\n'), file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        capture.output(kappa2(df_[,c('obsZone','trackZone')], "unweighted") ,
                   file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        capture.output(agree(df_[,c('obsZone','trackZone')], tolerance=0) ,
                   file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        }
    
    ################ mismatch ################
    cat('\n------------------------------------------------------------------\n',
        file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat(paste0('-----------------',x,'Mismatch -------------------\n'),
        file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat('------------------------------------------------------------------\n',
        file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    #mismatch all pens
    cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat('-------all pen --------\n', file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)    #reliability all systems together
    #confusion matrix: prediccted than reference: predicted is trackobs
    capture.output(confusionMatrix(df$trackZone, df$obsZone),
                   file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
    #mismatch for each systems separately
    c_pens = list(c(3,4,5),c(8,9),c(10,11,12))
    for (c_pen in c_pens){
        df_ = subset(df, df$Pen %in% c_pen)
        cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        cat(paste0('pen: \n ',c_pen,'\n'), file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        cat('------------------------\n',file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        capture.output(confusionMatrix(df_$trackZone, df_$obsZone),
                   file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
        }        
}

Parsed with column specification:
cols(
  Date = col_character(),
  Time = col_time(format = ""),
  Pen = col_double(),
  Side = col_character(),
  Bird = col_character(),
  obsZone = col_character(),
  `Position / Comments` = col_character(),
  TrackingTag = col_double(),
  trackZone = col_character(),
  trackZone_automatic = col_character(),
  automatic_comment = col_character(),
  SecondsBefore = col_double(),
  SecondsAfter = col_double(),
  is_correct = col_logical(),
  Comments = col_character(),
  New_is_correct = col_character()
)

The following `from` values were not present in `x`: Tier 1, Tier 3 Rampe + Nestbox, Tier 2 (mini 12), Tier 2 (mini 11), Wintergarten, Tier 2 (mini 10), Tier 2 + untere Stange, Tier 4 + obere Stange, Tier 2 (mini 4), Tier 2 (mini 5), Tier 2 (mini 3), Tier 2 (mini 8), Tier 2 (mini 9), Tier 2, Tier 1, Tier 3, Tier 4, 1, 2, 3, 4, Wg, N/A, NA

The following `from` values were not present in `x`: Tier 1, Tier 3 Rampe + Nestbox, Tier 2 (mini 12), Tier 2 (m

[1] 215  16
[1] 204  16


Parsed with column specification:
cols(
  Date = col_character(),
  Pen = col_double(),
  Tag = col_double(),
  ObservationAnalysis = col_double(),
  obsTime = col_character(),
  obsZone = col_character(),
  TrackingAnalysis = col_double(),
  trackTime = col_character(),
  trackZone = col_character(),
  realTrackZone = col_character(),
  Comments = col_character()
)

The following `from` values were not present in `x`: Tier 1, Tier 3 Rampe + Nestbox, Tier 2 (mini 12), Tier 2 (mini 11), Wintergarten, Tier 2 (mini 10), Tier 2 + untere Stange, Tier 4 + obere Stange, Tier 2 (mini 4), Tier 2 (mini 5), Tier 2 (mini 3), Tier 2 (mini 8), Tier 2 (mini 9), Tier 2, Tier 1, Tier 3, Tier 4, NA

The following `from` values were not present in `x`: Tier 1, Tier 3 Rampe + Nestbox, Tier 2 (mini 12), Tier 2 (mini 11), Wintergarten, Tier 2 (mini 10), Tier 2 + untere Stange, Tier 4 + obere Stange, Tier 2 (mini 4), Tier 2 (mini 5), Tier 2 (mini 3), Tier 2 (mini 8), Tier 2 (mini 9), Tier 2, Tier 1, Tier 3, 

[1] 215  11
[1] 208  11


Parsed with column specification:
cols(
  Date = col_character(),
  batch = col_double(),
  Heure = col_time(format = ""),
  TagID = col_character(),
  BirdID = col_double(),
  trackZone = col_character(),
  signalstrength = col_double(),
  VideoAnalyse = col_double(),
  Comment = col_character(),
  worker = col_character(),
  Pen = col_double(),
  backpackID = col_character(),
  Obszone_whenWrong = col_character(),
  obsZone = col_character()
)

The following `from` values were not present in `x`: Tier 1, Tier 3, 1, 2, 3, 4, Wg, N/A

The following `from` values were not present in `x`: Tier 2, Tier 1, Tier 3, Tier 4, 1, 2, 3, 4, Wg, N/A



[1] 1025   14
[1] 923  14
[1] 923  14
[1] 869  14
[1] 638  14


# Time lag

In [6]:
df_focals = read_delim(file.path(path_records,'ValidationAllFocals.csv'),';',trim_ws = TRUE)
df_focals[c('obsZone','trackZone')] = lapply(df_focals[c('obsZone','trackZone')], factor)
print(dim(df_focals))
head(df_focals,3)

Parsed with column specification:
cols(
  Date = col_character(),
  Pen = col_double(),
  Tag = col_double(),
  ObservationAnalysis = col_double(),
  obsTime = col_character(),
  obsZone = col_character(),
  TrackingAnalysis = col_double(),
  trackTime = col_character(),
  trackZone = col_character(),
  realTrackZone = col_character(),
  Comments = col_character()
)



[1] 215  11


Date,Pen,Tag,ObservationAnalysis,obsTime,obsZone,TrackingAnalysis,trackTime,trackZone,realTrackZone,Comments
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<chr>,<fct>,<chr>,<chr>
31.07.2020,10,22,1,15:08:27,Wg,1,15:09:13,Wg,Wintergarten,delay 00:00:46
31.07.2020,10,22,1,15:09:41,1,1,15:09:58,1,Tier 1,NA
31.07.2020,10,22,1,15:16:55,Wg,1,15:22:22,Wg,Wintergarten,delay 00:05:27


In [7]:
df_focals = subset(df_focals, df_focals$obsTime != "N/A")
df_focals = subset(df_focals, df_focals$trackTime != "N/A")
df_focals$obsTime = as.POSIXct(df_focals$obsTime,format="%H:%M:%S")
df_focals$trackTime = as.POSIXct(df_focals$trackTime,format="%H:%M:%S")
# time difference in seconds between Tracking and Obs
df_focals = mutate(df_focals, timediff=difftime(df_focals$trackTime, df_focals$obsTime ,units="secs"))
df_focals$timediff = as.numeric(df_focals$timediff)

### all pens together

In [8]:
#info
cat('------------------- percentage of data within the 2s frame (<= 2 / >= -2)\n',
    file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
cat(100/nrow(df_focals)*nrow(subset(df_focals, df_focals$timediff<= 2 & df_focals$timediff >= -2)),
    file=file.path(path_analysis,'GantnerAnalysisSummary.txt'), append=TRUE)
#paste(max(df_focals$timediff), min(df_focals$timediff), mean(df_focals$timediff), median(df_focals$timediff), sep='   ')

In [9]:
#histogram
png(file.path(path_analysis,'TimeLagHistogram_allpens.png')) #TODO: height=2, width=8
hist(df_focals$timediff, main="Histogram of time difference all pens", breaks=20)
dev.off()

png 
  2

In [10]:
png(file.path(path_analysis,'TimeLagBoxplot_allpens.png')) #TODO: height=2, width=8
boxplot(df_focals$timediff, staplewex=1)
text(y=fivenum(df_focals$timediff), labels=fivenum(df_focals$timediff), x=1.25)
title('Boxplot of time difference all pens')
dev.off()

png 
  2

### for each pen

In [11]:
#histogram
df_focals$timediff = as.numeric(df_focals$timediff)
png(file.path(path_analysis,'TimeLagHistoGram.png')) #TODO: height=2, width=8
layout(mat = matrix(c(1,2,3,4,5,6,7,8), nrow=2, ncol=4),
       heights = c(1,1), # Heights of the rows
       widths = c(3,3,3,3)) # Widths of the columns
for (i in unique(df_focals$Pen)){
    df_focals_ = subset(df_focals, df_focals$Pen==i)
    #ylim=c(max(df_focals$timediff), min(df_focals$timediff))
    hist(df_focals_$timediff, breaks=20, main=paste0('Pen ',i))
}
#1=bottom, 2=left, 3=top, 4=right
#mtext("Time difference per pen", side=3, line=0, outer=TRUE)#doe snot work
dev.off()

png 
  2

In [12]:
png(file.path(path_analysis,'TimeLagBoxplot.png')) #TODO: height=2, width=8
layout(mat = matrix(c(1,2,3,4,5,6,7,8), nrow=2, ncol=4),
       heights = c(1,1), # Heights of the rows
       widths = c(2,2,2,2)) # Widths of the columns
for (i in unique(df_focals$Pen)){
    df_focals_ = subset(df_focals, df_focals$Pen==i)
    #ylim=c(max(df_focals$timediff), min(df_focals$timediff))
    boxplot(df_focals_$timediff, staplewex=1)
    text(y=fivenum(df_focals_$timediff), labels=fivenum(df_focals_$timediff), x=1.25)
    title(paste0('Pen ',i))
}
dev.off()

png 
  2

In [13]:
sink()